# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902079


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/111 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/111 [00:02<02:33,  1.41s/it]

Rendering models:   4%|▎         | 4/111 [00:04<02:09,  1.21s/it]

Rendering models:   5%|▌         | 6/111 [00:04<01:30,  1.15it/s]

Rendering models:   8%|▊         | 9/111 [00:04<01:03,  1.61it/s]

...

Rendering models:  86%|████████▋ | 96/111 [00:12<00:00, 18.37it/s]

Rendering models:  92%|█████████▏| 102/111 [00:12<00:00, 21.97it/s]

Rendering models:  95%|█████████▌| 106/111 [00:12<00:00, 24.48it/s]

Rendering models:  98%|█████████▊| 109/111 [00:12<00:00, 23.25it/s]

equidad1                              0.000383
kayleebug2017                         0.001793
DrJackie                              0.000980
Thedunkmasta                          0.001720
Thedunkmasta                          0.001626
Thedunkmasta                          0.001825
Thedunkmasta                          0.001809
Thedunkmasta                          0.002383
Thedunkmasta                          0.001943
Thedunkmasta                          0.002040
Thedunkmasta                          0.001756
Thedunkmasta                          0.001722
DavidCastillo1702                     0.001012
not-logged-in-769e51a01946aca9871e    0.569979
sinjinza1984                          0.000459
graciegirl                            0.000534
Michidisperso                         0.000410
Paddy_van_CH                          0.000490
not-logged-in-a0164d241f30411408f0    0.000782
clizardo                              0.000454
mkirms                                0.000859
clizardo     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())